# Final Project: Yelp and Food Safety
#### Exploring the San Francisco Restaurant World

In this project, we will investigate a subset of the restaurants on Yelp located in San Francisco, California. You will first explore some of the data about the restaurants themselves, calculating some summary statistics and trying to figure out some patterns in the data. Next we will merge that with a list of health inspection scores and violations that have been [made available by the San Francisco Department of Public Health](https://data.sfgov.org/Health-and-Social-Services/Restaurant-Scores-LIVES-Standard/pyih-qa8i). Lastly, we will run some linear regression analysis to see if there is any meaningful relationship between health inspection scores, and other aspects of each restaurant

If you have any questions or get stuck or anything, feel free to come to office hours. 

**Helpful Resource:**

* [Python Reference](https://docs.google.com/document/d/1zpTTl47NoGf2A3_oE1YusLyb-cF2sZMALdCMM5dpYIA/edit): Cheat sheet for Python and other functions used in this course


To get started on the final project, run the following cell to download the necessary materials, and have fun! 


In [1]:
# importing some helful libraries
import pandas as pd
import numpy as np
from functions import * 

# downloading necessary data 

# !wget 


## 1. San Franciso Restaurant Data

In this final project you'll be learning a few more fancy ways to deal with _dataframes_, which we previously introduced in labs 3 and 4 for holding and managing data to be used in Python. The package that is most commonly used to handle data analysis and manipulation in Python is called _Pandas_. 

Before we get into anything else about Pandas, let's first go over the data we will be using

- `businesses.csv` is a file that contains information for a little over 700 restaurants in San Francisco. Each restaurant has a unique business id, which we will make use of.  
- `inspections.csv` is a file that contains a list of health inspections conducted by the San Francisco Department of Public Health

Most of the columns in the Dataframes are rather self-explanatory. However, the `bid` column stands for the _*business ID*_